In [ ]:
%run ./../data/load-dataset.ipynb
%run ./../homegrown/basic-neural-network.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
ITERS = 100

RANDOM_SEED = 0

In [ ]:
def sparse_to_dense(sparse_bow_doc, dict_size):
    dense_bow_doc = np.zeros(dict_size).astype(int)
    dense_bow_doc[sparse_bow_doc.T[0]] = sparse_bow_doc.T[1]
    return dense_bow_doc


corpus, labels = df[[proc_doc_col, label_col]].T.values
X = np.array([sparse_to_dense(np.array(dictionary.doc2bow(doc)), len(dictionary)) for doc in corpus])
y = np.array([np.eye(2)[int(label)] for label in labels])
train_samples, test_samples, train_labels, test_labels = train_test_split(X, y, random_state=RANDOM_SEED)

In [ ]:
hidden_layer_size = 100
iterations = ITERS + 1
alpha = 1e-3
batch_size = 512  # set to 1 for stochastic gradient descent
hid_activation = Activation.RELU
out_activation = Activation.SIGMOID
dropout = False
as_probs = True
skip_remaining = True
yield_results = True
silent = False
random_seed = RANDOM_SEED

model = BasicNeuralNetwork(hidden_layer_size, epochs=iterations, alpha=alpha,
                           batch_size=batch_size, skip_remaining=skip_remaining,
                           hid_activation=hid_activation, out_activation=out_activation, 
                           dropout=dropout, as_probs=as_probs, 
                           yield_results=yield_results, silent=silent, random_seed=random_seed)

data = []
predict_sentiment_gen = model.evaluate(train_samples, test_samples, train_labels, test_labels)
for (train_preds, train_errors), (test_preds, test_errors) in predict_sentiment_gen:
    train_loss = sum(train_errors) / len(train_samples)
    train_correct = sum([np.argmax(pred) == np.argmax(label)
                         for pred, label
                         in zip(train_preds, train_labels)])
    train_accuracy = train_correct / len(train_samples)
    
    test_loss = sum(test_errors) / len(test_samples)
    test_correct = sum([np.argmax(pred) == np.argmax(label)
                        for pred, label
                        in zip(test_preds, test_labels)])
    test_accuracy = test_correct / len(test_samples)
    
    data.append({'Train Predictions': train_preds, 'Train Errors': train_errors,
                 'Train Loss (MSE)': train_loss, 'Train Accuracy': train_accuracy,
                 'Test Predictions': test_preds, 'Test Errors': test_errors,
                 'Test Loss (MSE)': test_loss, 'Test Accuracy': test_accuracy,
                 'Correct (Train)': train_correct, 'Correct (Test)': test_correct})

results = pd.DataFrame(data)
results[['Train Loss (MSE)', 'Train Accuracy', 'Test Loss (MSE)', 'Test Accuracy']][::10].style \
            .highlight_min(subset=['Train Accuracy', 'Test Accuracy'], color='lightcoral') \
            .highlight_max(subset=['Train Accuracy', 'Test Accuracy'], color='lightgreen') \
            .highlight_min(subset=['Train Loss (MSE)', 'Test Loss (MSE)'], color='lightgreen') \
            .highlight_max(subset=['Train Loss (MSE)', 'Test Loss (MSE)'], color='lightcoral')

In [ ]:
normalize = 'true'

true_preds, true_labels = [np.argmax(pred) for pred in test_preds], \
                          [np.argmax(label) for label in test_labels]
ConfusionMatrixDisplay.from_predictions(true_labels, true_preds, normalize=normalize,
                                        cmap=plt.cm.Blues, display_labels=('reliable', 'unreliable'))